In [ ]:
import sqlite3
import sys

# Connect to the SQLite database
con = sqlite3.connect("StudentDB")
curs = con.cursor()

# Function to create the PersonTB table
def createPersonTable():
    """
    Create a table named PersonTB if it doesn't already exist.

    The table has columns: roll_no (integer, primary key), Name (text), and Age (integer).
    """
    curs.execute("CREATE TABLE IF NOT EXISTS PersonTB(roll_no INTEGER PRIMARY KEY, Name TEXT, Age INTEGER)")
    print("Table created")

# Function to create the AcademicTB table
def createAcademicTable():
    """
    Create a table named AcademicTB if it doesn't already exist.

    The table has columns: roll_no (integer, auto-increment primary key), Semester (integer), Branch (text),
    Marks (integer), and Percentile (float).
    """
    curs.execute("CREATE TABLE IF NOT EXISTS AcademicTB(roll_no INTEGER PRIMARY KEY AUTOINCREMENT, "
                 "Semester INTEGER, Branch TEXT, Marks INTEGER, Percentile REAL)")
    print("Academic Table Created")

# Function to insert a single academic record into AcademicTB
def insertAcademicRecord(tup1):
    """
    Insert a record into the AcademicTB table.

    Args:
        tup1 (tuple): A tuple containing values for Semester, Branch, and Marks.

    If there is an integrity error (e.g., duplicate primary key), it will be caught and displayed.
    """
    try:
        curs.execute("INSERT INTO AcademicTB(Semester, Branch, Marks) VALUES(?,?,?)", tup1)
        con.commit()
        print("The information is successfully entered")
    except sqlite3.IntegrityError as e:
        print(e)
# Function to insert a single person record into PersonTB
def insertPerson1Record(tup1):
    """
    Insert a record into the PersonTB table.

    Args:
        tup1 (tuple): A tuple containing values for roll_no, Name, and Age.

    If there is an integrity error (e.g., duplicate primary key), it will be caught and displayed.
    """
    try:
        curs.execute("INSERT INTO PersonTB(roll_no, Name, Age) VALUES(?,?,?)", tup1)
        con.commit()
        print("The information is successfully entered")
    except sqlite3.IntegrityError as e:
        print(e)

# Function to show all records in AcademicTB
def showAllRecordsInAcademicTB():
    """
    Display all records in the AcademicTB table.
    """
    curs.execute("SELECT * FROM AcademicTB")
    rows = curs.fetchall()
    if len(rows) == 0:
        print("No Record")
        return
    print("Roll NO\t\tSemester\tBranch\tMarks\tPercentile")
    for row in rows:
        # Print each row in a formatted manner
        print(row[0], "\t\t\t\t", row[1], "\t\t", row[2], "\t", row[3], "\t", row[4])

# Function to calculate percentile and update the AcademicTB
def calculatePercentile():
    """
    Calculate and update the percentile for each student in the AcademicTB table.

    This function retrieves the maximum Marks from AcademicTB and then calculates the percentile
    for each student based on their Marks and updates the Percentile column in the table.
    """
    curs.execute("SELECT MAX(Marks) FROM AcademicTB")
    row = curs.fetchone()
    maxMarks = row[0]

    curs.execute("SELECT roll_no, Marks FROM AcademicTB")
    rows = curs.fetchall()
    for row in rows:
        roll = row[0]
        marks = row[1]
        percentile = (marks / maxMarks) * 100
        curs.execute("UPDATE AcademicTB SET Percentile=? WHERE roll_no=?", (percentile, roll))
        con.commit()

# Function to insert multiple person records into PersonTB
def insertPersonManyRecords(lst):
    """
    Insert multiple records into the PersonTB table.

    Args:
        lst (list): A list of tuples, each containing values for roll_no, Name, and Age.

    If there is an integrity error (e.g., duplicate primary key), it will be caught and displayed.
    """
    try:
        curs.executemany("INSERT INTO PersonTB(roll_no, Name, Age) VALUES(?,?,?)", lst)
        con.commit()
        print("The information is successfully entered")
    except sqlite3.IntegrityError as e:
        print(e)

# Function to show all records in PersonTB
def showAllPersonRecords():
    """
    Display all records in the PersonTB table.
    """
    curs.execute("SELECT * FROM PersonTB")
    rows = curs.fetchall()
    if len(rows) == 0:
        print("No Record")
        return
    print("Roll NO\t\tNAME\tAGE")
    for row in rows:
        print(row[0], "\t\t", row[1], "\t", row[2])

# Function to show a person record by roll number
def showPersonRecordByRollno(roll):
    """
    Display a record from the PersonTB table based on the provided roll number.

    Args:
        roll (int): The roll number to search for.

    Returns:
        tuple: A tuple containing roll_no, Name, and Age if found, or None if not found.
    """
    curs.execute("SELECT * FROM PersonTB WHERE roll_no=?", (roll,))
    row = curs.fetchone()
    if row is None:
        print("No record for this roll number")
        return None

    print("Roll NO\t\tNAME\tAGE")
    print(row[0], "\t\t", row[1], "\t", row[2])
    return row

# Function to delete a person record by roll number
def delPersonRecordByRollNo(roll):
    """
    Delete a record from the PersonTB table based on the provided roll number.

    Args:
        roll (int): The roll number of the record to be deleted.

    This function checks if the provided roll number exists in the table.
    If it's not found, it prints an error message. Otherwise, it deletes the record.
    """
    curs.execute("SELECT ROLL_NO FROM PersonTB WHERE ROLL_NO = ?", (roll,))
    existing_record = curs.fetchone()

    if existing_record is None:
        print("Enter a valid roll number")
        return

    curs.execute("DELETE FROM PersonTB WHERE ROLL_NO = ?", (roll,))
    con.commit()
    print("Record is successfully deleted")


# Function to fetch all roll numbers from PersonTB
def fetchAllRollno():
    """
    Fetch all roll numbers from the PersonTB table.

    Returns:
        list: A list of roll numbers.
    """
    curs.execute("SELECT ROLL_NO FROM PersonTB")
    rows = curs.fetchall()
    return rows

# Function to delete all records in PersonTB
def deleteAllRecords():
    """
    Delete all records from the PersonTB table.
    """
    curs.execute("DELETE FROM PersonTB")
    con.commit()
    print("All records are deleted, the table is now empty")

# Function to show a person record by name
def showRecordByName(Name):
    """
    Display a record from the PersonTB table based on the provided Name.

    Args:
        Name (str): The name to search for in the table.

    Returns:
        tuple: A tuple containing roll_no, Name, and Age if found, or None if not found.
    """
    curs.execute("SELECT * FROM PersonTB WHERE NAME=?", (Name,))
    row = curs.fetchone()
    if row is None:
        print("No record for this Name")
        return None

    print("Roll NO\t\tNAME\tAGE")
    print(row[0], "\t\t", row[1], "\t", row[2])
    return row

# Function to show all records in PersonTB by age in descending order
def showRecordsByAge():
    """
    Display all records from the PersonTB table ordered by Age in descending order.
    """
    curs.execute("SELECT * FROM PERSONTB ORDER BY Age DESC")
    rows = curs.fetchall()
    if rows == None:
        print("No record for this age")
        return

    print("Roll NO\t\tNAME\tAGE")
    for row in rows:
        print(row[0], "\t\t", row[1], "\t", row[2])

# Function to show records in PersonTB by age range
def showRecordsByAgeRange(minAge, maxAge):
    """
    Display records from the PersonTB table within a specified age range.

    Args:
        minAge (int): The minimum age in the range.
        maxAge (int): The maximum age in the range.
    """
    # curs.execute("SELECT * FROM PERSONTB WHERE Age >= ? AND Age <= ?", (minAge, maxAge))
    curs.execute("SELECT * FROM PERSONTB WHERE Age BETWEEN ? AND ?", (minAge, maxAge))

    rows = curs.fetchall()
    if len(rows) == 0:
        print("No record in this range")
        return

    print("Roll NO\t\tNAME\tAGE")
    for row in rows:
        print(row[0], "\t\t", row[1], "\t", row[2])

# Function to show records in PersonTB by name and age in descending order
def showRecordsByNameAge():
    """
    Display all records from the PersonTB table ordered by Name and then by Age in descending order.
    """
    curs.execute("SELECT * FROM PERSONTB ORDER BY NAME, Age DESC")
    rows = curs.fetchall()
    if rows == None:
        print("No record for this age")
        return

    print("Roll NO\t\tNAME\tAGE")
    for row in rows:
        print(row[0], "\t\t", row[1], "\t", row[2])

# Function to count all records in PersonTB
def countAllRecords():
    """
    Count and display the total number of records in the PersonTB table.
    """
    curs.execute("SELECT COUNT(*) FROM PERSONTB")
    row = curs.fetchone()
    if row == None:
        print("No record in table")
        return
    print("No Of Records Are =", row[0])

# Function to count records in PersonTB within a given age range
def countNoOfRecorsInAGivenRange(minAge, maxAge):
    """
    Count and display the number of records within a specified age range in the PersonTB table.

    Args:
        minAge (int): The minimum age in the range.
        maxAge (int): The maximum age in the range.
    """
    curs.execute("SELECT COUNT(*) FROM PERSONTB WHERE Age BETWEEN ? AND ?", (minAge, maxAge))

    row = curs.fetchone()
    if len(row) == 0:
        print("No record in this range")
        return
    print("No Of Records Are =", row[0])

# Function to show min age
def showMinAge():
    """
    Display the minimum age found in the PersonTB table.
    """
    curs.execute("SELECT MIN(AGE) FROM PERSONTB")
    row = curs.fetchone()
    if row == None:
        print("No record in table")
        return
    print("Min age is =", row[0])

if __name__=="__main__":
    try:
        while True:
            print("1. Create Personal Table\n2. Insert 1 Record In PersonalTB\n"
                  "3. Insert Many Records in PersonalTB\n4. Display All Records\n5. Display Record By Roll No"
                  "\n6. Show Record By Name"
                  "\n7. Show Records In Descending Order"
                  "\n8. Show Records By Age Range"
                  "\n10. Delete Person Record By Roll No"
                  "\n11. Delete All Records"
                  "\n15. Update"
                  "\n16. Count All Records"
                  "\n17. Count Records In A Given Age Range"
                  "\n21. Create academic table"
                  "\n22. Insert 1 Record in Academic Table"
                  "\n23. Display All Records In AcademicTB"
                  "\n24. Calculate Percentile"
                  "\n30. Exit")
            print("=" * 30)
            choice = int(input("Enter your choice ="))
            if choice == 1:
                createPersonTable()
            elif choice == 2:
                roll_no = int(input("Enter the roll_no ="))
                Name = input("Enter the Name =")
                Age = int(input("Enter the Age ="))
                tup1 = (roll_no, Name, Age)
                insertPerson1Record(tup1)
            elif choice == 3:
                n = int(input("Enter the number of records you want to give ="))
                lst = []
                for i in range(n):
                    roll_no = int(input("Enter the roll_no ="))
                    Name = input("Enter the Name =")
                    Age = int(input("Enter the Age ="))
                    tup1 = (roll_no, Name, Age)
                    lst.append(tup1)
                insertPersonManyRecords(lst)
            elif choice == 4:
                showAllPersonRecords()
            elif choice == 5:
                roll = int(input("Enter the roll number ="))
                showPersonRecordByRollno(roll)
            elif choice == 6:
                Name = input("Enter the Name =")
                showRecordByName(Name)
            elif choice == 7:
                showRecordsByAge()
            elif choice == 8:
                minAge = int(input("Enter the min age"))
                maxAge = int(input("Enter the max age"))
                showRecordsByAgeRange(minAge, maxAge)
            elif choice == 10:
                roll = int(input("Enter the roll number ="))
                delPersonRecordByRollNo(roll)
            elif choice == 11:
                deleteAllRecords()
            elif choice == 15:
                roll = int(input("Enter the roll number ="))
                updatePersonRecordByRollNo(roll)
            elif choice == 16:
                countAllRecords()
            elif choice == 17:
                minAge = int(input("Enter the min age"))
                maxAge = int(input("Enter the max age"))
                countNoOfRecorsInAGivenRange(minAge, maxAge)
            elif choice == 18:
                showMinAge()
            elif choice == 21:
                createAcadamicTable()
            elif choice == 22:
                sem = int(input("Enter the semester ="))
                branch = input("Enter the branch =")
                marks = int(input("Enter the marks ="))
                tup1 = (sem, branch, marks)
                insertAcademicRecord(tup1)
            elif choice == 23:
                showAllRecordsInAcademicTB()
            elif choice == 24:
                calculatePercentile()
            elif choice == 30:
                sys.exit()
            print("=" * 45)
    except ValueError as e:
        print("Please enter in correct format")

        
#work on names 
#show record by name======= DONE
#show records by age range
#arrange records in decending order by age
# for second question --using range and using between
#AVG SUM MAX DO IT IN CONDITIONS (range pe where pe etc)
#DISTINCT KEYWORD --GERENRALLY NEEDED TO ONLY PRINT A RECORD WITH SIMILAR DATA ONCE
#DROP TABLE




1. Create Personal Table
2. Insert 1 Record In PersonalTB
3. Insert Many Records in PersonalTB
4. Display All Records
5. Display Record By Roll No
6. Show Record By Name
7. Show Records In Descending Order
8. Show Records By Age Range
10. Delete Person Record By Roll No
11. Delete All Records
15. Update
16. Count All Records
17. Count Records In A Given Age Range
21. Create academic table
22. Insert 1 Record in Academic Table
23. Display All Records In AcademicTB
24. Calculate Percentile
30. Exit
Enter your choice =4
Roll NO		NAME	AGE
1 		 vishal 	 24
2 		 garvit 	 25
3 		 shreyas 	 34
4 		 shantanu 	 36
1. Create Personal Table
2. Insert 1 Record In PersonalTB
3. Insert Many Records in PersonalTB
4. Display All Records
5. Display Record By Roll No
6. Show Record By Name
7. Show Records In Descending Order
8. Show Records By Age Range
10. Delete Person Record By Roll No
11. Delete All Records
15. Update
16. Count All Records
17. Count Records In A Given Age Range
21. Create academic ta